In [6]:
import tensorflow as tf
import threading, cv2, os

In [7]:
# Create a dummy lists for image files and associated annotations
file_names = [
    '/Users/aponamaryov/Downloads/coco_train_2014/images/COCO_train2014_000000000009.jpg',
    '/Users/aponamaryov/Downloads/coco_train_2014/images/COCO_train2014_000000000025.jpg',
    '/Users/aponamaryov/Downloads/coco_train_2014/images/COCO_train2014_000000000030.jpg',
    '/Users/aponamaryov/Downloads/coco_train_2014/images/COCO_train2014_000000000034.jpg',
    '/Users/aponamaryov/Downloads/coco_train_2014/images/COCO_train2014_000000000036.jpg',
    '/Users/aponamaryov/Downloads/coco_train_2014/images/COCO_train2014_000000000049.jpg',
    '/Users/aponamaryov/Downloads/coco_train_2014/images/COCO_train2014_000000000061.jpg',
    '/Users/aponamaryov/Downloads/coco_train_2014/images/COCO_train2014_000000000064.jpg',
    '/Users/aponamaryov/Downloads/coco_train_2014/images/COCO_train2014_000000000071.jpg',
    '/Users/aponamaryov/Downloads/coco_train_2014/images/COCO_train2014_000000000072.jpg'
]

anns = [
    {'bboxes': [[10.0,15.0,7.0,12.0],[12.0,17.0,9.0,14.0],[14.0,19.0,5.0,14.0]], 'labels': [1,2,1]},
    {'bboxes': [[10.0,15.0,7.0,12.0],[12.0,17.0,9.0,14.0],[14.0,19.0,5.0,14.0]], 'labels': [2,2,1]},
    {'bboxes': [[10.0,15.0,7.0,12.0],[12.0,17.0,9.0,14.0],[14.0,19.0,5.0,14.0]], 'labels': [3,2,1]},
    {'bboxes': [[10.0,15.0,7.0,12.0],[12.0,17.0,9.0,14.0],[14.0,19.0,5.0,14.0]], 'labels': [4,2,1]},
    {'bboxes': [[10.0,15.0,7.0,12.0],[12.0,17.0,9.0,14.0],[14.0,19.0,5.0,14.0]], 'labels': [5,2,1]},
    {'bboxes': [[10.0,15.0,7.0,12.0],[12.0,17.0,9.0,14.0],[14.0,19.0,5.0,14.0]], 'labels': [6,2,1]},
    {'bboxes': [[10.0,15.0,7.0,12.0],[12.0,17.0,9.0,14.0],[14.0,19.0,5.0,14.0]], 'labels': [7,2,1]},
    {'bboxes': [[10.0,15.0,7.0,12.0],[12.0,17.0,9.0,14.0],[14.0,19.0,5.0,14.0]], 'labels': [8,2,1]},
    {'bboxes': [[10.0,15.0,7.0,12.0],[12.0,17.0,9.0,14.0],[14.0,19.0,5.0,14.0]], 'labels': [9,2,1]},
    {'bboxes': [[10.0,15.0,7.0,12.0],[12.0,17.0,9.0,14.0],[14.0,19.0,5.0,14.0]], 'labels': [10,2,1]},
]

In [8]:
# Define a function that will retrieve one example
def retrieve_example(path, annotations, size=(640,640)):
    # 1. Check that provided path exists
    assert os.path.exists(path), "read method receivecd incorrect path. The following path doesn't exist {}".\
        format(path)
    # 2. Read the image
    im = cv2.imread(path, cv2.IMREAD_COLOR)
    b, g, r = cv2.split(im)
    im = cv2.merge([r, g, b])
    im = cv2.resize(im, size)
    bboxes, labels = annotations['bboxes'], annotations['labels']
    return [im, bboxes, labels]

example = retrieve_example(file_names[0], anns[0])
print "resulting image shape:", example[0].shape
print "bbox shape:",  len(example[1]), len(example[1][0])
print "number of labels", len(example[2])

resulting image shape: (640, 640, 3)
bbox shape: 3 4
number of labels 3


In [9]:
# create placeholders for data input
tf.reset_default_graph() # as always, reset the graph to ensure no prior run residuals will affect your script
img_input = tf.placeholder(tf.float32, shape=[640,640,3], name='img_input')
bbox_input = tf.placeholder(tf.float32, shape=[3,4], name='bbox_input')
label_input = tf.placeholder(tf.int16, shape=[3], name='labels_input')
# create a queue
queue = tf.FIFOQueue(capacity=10, dtypes=[tf.float32, tf.float32, tf.int16],
                    shapes=[[640,640,3], [3,4], [3]])
# define enqueue and dequeue operations
enqueue_inputs_op = queue.enqueue([img_input, bbox_input, label_input])
deque_inputs_op = queue.dequeue()

# define a function that will help to enqueue your inputs
def enqueue(sess):
    example = retrieve_example(file_names.pop(0), anns.pop(0))
    print example[2][0]
    sess.run(enqueue_inputs_op,
             feed_dict={
                 img_input: example[0],
                 bbox_input: example[1],
                 label_input: example[2]
             })
    
# define a batch reading operation
img_batch, bbox_batch, labels_batch = tf.train.batch(deque_inputs_op, batch_size=2, capacity=8)

In [10]:
with tf.Session() as sess:
    # enqueue 5 examples asynchronously
    for _ in xrange(5):
        enqueue_example = threading.Thread(target=enqueue, args=[sess])
        enqueue_example.isDaemon()
        enqueue_example.start()
    # enqueue remaining 5 examples sequentially
    for _ in xrange(5):
        enqueue(sess)
    # Create a coordinator that will run a tensorflow queue
    coord = tf.train.Coordinator()
    # start the queue runner with the coordinator
    threads = tf.train.start_queue_runners(coord=coord)
    img_cur_batch, bbox_cur_batch, labels_cur_batch = sess.run([img_batch, bbox_batch, labels_batch])
    print "dequeued imag batch has the following shape:", img_cur_batch.shape
    print "dequeued bbox batch has the following shape:", bbox_cur_batch.shape
    print "dequeued lables (with shape {}):".format(labels_cur_batch.shape)
    print labels_cur_batch
    # once youre done, close the queue
    queue.close(cancel_pending_enqueues=True)
    # stop the coordinator
    coord.request_stop()
    # merge the result of all the threads
    coord.join(threads)
    
print "successful execution"

2
361


4
5
7
8
9
10
dequeued imag batch has the following shape: (2, 640, 640, 3)
dequeued bbox batch has the following shape: (2, 3, 4)
dequeued lables (with shape (2, 3)):
[[2 2 1]
 [3 2 1]]
successful execution
